In [1]:
import pandas as pd
import requests
import json

## To get All Owners of the NFT

In [64]:
headers = {
    'accept': 'application/json',
    'X-API-Key': 'pkhgpCPTU5AFiwXHrTwNCVXBIPfS3HZBLpOp1YOc8SOKxA8dUSVCEt9sg8hFjokU',
}

params = {
    'chain': 'eth',
    'format': 'decimal',
}

holder_list = requests.get('https://deep-index.moralis.io/api/v2/nft/0x90cfCE78f5ED32f9490fd265D16c77a8b5320Bd4/owners', headers=headers, params=params)

# with open('Holder_address.json', 'w') as outfile:
#     outfile.write(response.text)

## Get the Balance in the wallet of each owners

In [65]:
headers = {
    'accept': 'application/json',
    'X-API-Key': 'pkhgpCPTU5AFiwXHrTwNCVXBIPfS3HZBLpOp1YOc8SOKxA8dUSVCEt9sg8hFjokU',
}

params = {
    'chain': 'eth',
}

owner_list = []
owner_result = []
for result in json.loads(holder_list.text)["result"]:
    if(len(owner_list)<20):
        owner_list.append(result["owner_of"])
    else:
        owner_addr = ",".join(owner_list)
        url = f"https://api.etherscan.io/api?module=account&action=balancemulti&address={owner_addr}&tag=latest&apikey=57GBAHGQSE3CMYNZQY81V1N32YJZNCCUNZ"
        response = requests.get(url, verify=True)
        result = json.loads(response.text)["result"]
        owner_result.extend(result)
        owner_list = []

In [66]:
pd_owner_result = pd.DataFrame(owner_result)

In [67]:
owner_value_count = pd_owner_result.value_counts(subset="account").to_frame().reset_index()
owner_value_count.columns = ["account", "counts"]
pd_unique_owner = pd_owner_result.drop_duplicates(ignore_index=True)

pd_owner = pd_unique_owner.merge(owner_value_count, on="account")
pd_owner["balance"] = pd_owner["balance"].astype(float)/1e18

# pd_owner.sort_values(by="counts")
# pd_owner.sort_values(by="balance")

## Count Holders with balance under threshold

In [68]:
threshold = 1
under_count = pd_owner[pd_owner["balance"]<=threshold]
under_count

,account,balance,counts
0,0x2b71a490d9e16a18f2debdc312ca41de460067fd,0.106385,1
1,0x0cb91dc1cbd4c11f5d303f381a922bfac4a5e46b,0.798309,1
4,0xfcddd485b3d388f84134c688818a927926d8d07c,0.300000,2
5,0x64a1a5c0d1ae79f1dcc6658fb4a3f15e78a0cdc3,0.089864,2
6,0x575396d861fd4dbbb168168dda8cfb91e6c2d16e,0.498000,1
...,...,...,...
388,0x24b06977997000c7916d635adbee70c145204d53,0.010000,1
389,0x1c5ee91f58fb0408587247b5bbe110cbc93c013c,0.014202,1
390,0xb53f8a839d5cbef30bc0b96e2010839ea03a9eb5,0.000000,1
391,0xe71bac1ce4ebf269c3794f44d136b34b85348568,0.222724,1


In [69]:
threshold = 20
over_count = pd_owner[pd_owner["balance"]>threshold]
over_count

,account,balance,counts
22,0x89f59ea4469d7b1252c8e5511ff2baa910504824,779.805012,9
118,0x7d8c6a6a640a2e69f4821559778e16648b9278b9,36.096789,1
169,0x373bf52192116c0fa431af3b6d0677a82549f34b,100.502635,1
172,0xf1cb971d30d1ca9d2d083ed1dd65f1a102216d79,45.323304,1
186,0xd8b07bc1bc3bae553bca5e94e99935dc12df24ff,40.708089,1


In [ ]:
# owner_result["balance"] = float(owner_result["balance"])/ 1e18
# with open('holders_balance.json', 'w') as outfile:
    # outfile.write(json.dumps(sort_result))

## Get all NFT's in the interest wallet

In [71]:
headers = {
    'accept': 'application/json',
    'X-API-Key': 'pkhgpCPTU5AFiwXHrTwNCVXBIPfS3HZBLpOp1YOc8SOKxA8dUSVCEt9sg8hFjokU',
}

params = {
    'chain': 'eth',
    'format': 'decimal',
}

In [93]:
interest_address = over_count["account"]
result_list = []
mapping = {}

""" Get all token of one account """
i = 0
for owner_addr in interest_address:
    url = f"https://deep-index.moralis.io/api/v2/{owner_addr}/nft"
    response = json.loads(requests.get(url, headers=headers, params=params).text)["result"]
    
    for r in response:
        if r["token_address"] in mapping.keys():
            result_list[mapping[r["token_address"]]]["Numbers"] += 1
        else:
            mapping[r["token_address"]] = i
            i += 1
            tmp = {
                'token_address': r["token_address"],
                'name': r["name"],
                'symbol': r["symbol"],
                'Numbers': 1,
            }
            result_list.append(tmp)

In [114]:
# Sort by collection Numbers
result_list = sorted(result_list, key=lambda r: r["Numbers"], reverse=True)
# result_list

## Add Lowest Prices to list

In [90]:
price_headers = {
    'accept': 'application/json',
    'X-API-Key': 'pkhgpCPTU5AFiwXHrTwNCVXBIPfS3HZBLpOp1YOc8SOKxA8dUSVCEt9sg8hFjokU',
}

price_params = {
    'chain': 'eth',
    'marketplace': 'opensea',
}

In [110]:
for r in result_list:
    price_url = f"https://deep-index.moralis.io/api/v2/nft/{r['token_address']}/lowestprice"
    price_response = requests.get(price_url, headers=price_headers, params=price_params)
    if price_response.status_code == 200:
        price_response = json.loads(price_response.text)
        price = float(price_response["price"])/ 1e18
        r['price'] = price
    else:
        r['price'] = 0

In [115]:
# Sort by value
result_list = sorted(result_list, key = lambda r: r['Numbers']*r['price'], reverse = True)
# result_list

## Get all ERC20 tokens in interest wallet

In [116]:
headers = {
    'accept': 'application/json',
    'X-API-Key': 'pkhgpCPTU5AFiwXHrTwNCVXBIPfS3HZBLpOp1YOc8SOKxA8dUSVCEt9sg8hFjokU',
}

params = {
    'chain': 'eth',
}

In [127]:
interest_address = over_count["account"]
result_list = []
mapping = {}

""" Get all token of one account """
i = 0
for owner_addr in interest_address:
    url = f"https://deep-index.moralis.io/api/v2/{owner_addr}/erc20"
    response = json.loads(requests.get(url, headers=headers, params=params).text)
    
    for r in response:
        if r["token_address"] in mapping.keys():
            result_list[mapping[r["token_address"]]]["Numbers"] += float(r["balance"])/ 10**float(r["decimals"])
        else:
            mapping[r["token_address"]] = i
            i += 1
            tmp = {
                'token_address': r["token_address"],
                'name': r["name"],
                'symbol': r["symbol"],
                'Numbers': float(r["balance"])/ 10**float(r["decimals"]),
            }
            result_list.append(tmp)

In [129]:
# Sort by collection Numbers
result_list = sorted(result_list, key=lambda r: r["Numbers"], reverse=True)
result_list

[{'token_address': '0x3b78fa671698e6a6fb7956937b64550d95966606',
  'name': 'NFT DAO',
  'symbol': 'NAO',
  'Numbers': 199999999999.99997},
 {'token_address': '0x66de58a5296be8f396f61be0c37df6ab55c561c2',
  'name': 'McDonalds DAO',
  'symbol': 'McDonalds',
  'Numbers': 117200474.8310153},
 {'token_address': '0x36c5a1f80a6a978aaff02a8b7f3d4b5a363632fb',
  'name': 'Google Metaverse',
  'symbol': 'Google',
  'Numbers': 117200474.8310153},
 {'token_address': '0xe10c7a38a537a861d2b2e87861af0edd3f2f58e1',
  'name': 'Degen DAO',
  'symbol': 'Degen',
  'Numbers': 117200474.8310153},
 {'token_address': '0x205c58815d158b38e42b979a81a740dfb2eff15c',
  'name': 'NASA Metaverse',
  'symbol': 'NASA',
  'Numbers': 117200474.8310153},
 {'token_address': '0xa04e18e86408fa651c144f10de5236493d070e78',
  'name': 'Amazon Metaverse',
  'symbol': 'Amazon',
  'Numbers': 117200474.8310153},
 {'token_address': '0x288c72aab7e6d1f688964c95404bc34dc0978528',
  'name': 'Apple metaverse',
  'symbol': 'APPLE',
  'Numbe

## Add Liquidity Prices to list

In [130]:
""" Lowest Prices"""
price_headers = {
    'accept': 'application/json',
    'X-API-Key': 'pkhgpCPTU5AFiwXHrTwNCVXBIPfS3HZBLpOp1YOc8SOKxA8dUSVCEt9sg8hFjokU',
}

price_params = {
    'chain': 'eth',
}

In [137]:
for r in result_list:
    price_url = f"https://deep-index.moralis.io/api/v2/erc20/{r['token_address']}/price"
    price_response = requests.get(price_url, headers=price_headers, params=price_params)
    if price_response.status_code == 200:
        price_response = json.loads(price_response.text)
        r['price'] = float(price_response["usdPrice"])
    else:
        r['price'] = 0

In [138]:
# Sort by value
result_list = sorted(result_list, key = lambda r: r['Numbers']*r['price'], reverse = True)
result_list

[{'token_address': '0x9416ba76e88d873050a06e5956a3ebf10386b863',
  'name': 'Virtue',
  'symbol': 'VIRTUE',
  'Numbers': 14.028333889445001,
  'price': 1.1385857258360461e+42},
 {'token_address': '0x3b78fa671698e6a6fb7956937b64550d95966606',
  'name': 'NFT DAO',
  'symbol': 'NAO',
  'Numbers': 199999999999.99997,
  'price': 0.000365556346623184},
 {'token_address': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48',
  'name': 'USD Coin',
  'symbol': 'USDC',
  'Numbers': 214118.42862,
  'price': 1.0},
 {'token_address': '0x1412eca9dc7daef60451e3155bb8dbf9da349933',
  'name': 'A68.net',
  'symbol': 'A68.net',
  'Numbers': 187978.56000000003,
  'price': 1.0834964417797455},
 {'token_address': '0xf9d25eb4c75ed744596392cf89074afaa43614a8',
  'name': 'Up1.org',
  'symbol': 'Up1.org',
  'Numbers': 226108.0,
  'price': 0.8531667780338583},
 {'token_address': '0xc7870abf8b0d927e40b96bc8125ed2644691b27e',
  'name': 'ApMoney.org',
  'symbol': 'ApMoney.org',
  'Numbers': 300000.0,
  'price': 0.5287378696

In [26]:
# interest_address = over_count["account"]
# result_list = []
# """ Get all token balance of one account """
# for owner_addr in interest_address:
#     url = f"https://api.ethplorer.io/getAddressInfo/{owner_addr}?showETHTotals=false?apiKey=EK-2MKYK-SE7nJY3-W5LCS"
#     response = requests.get(url, verify=True)
#     result_list.append(response.text)

In [56]:
# with open('Owner Token balance.json', 'w') as outfile:
#     outfile.write("[" + ", ".join(result_list) + "]")

# with open('Owner Token balance.json', 'r') as f:
#     result = json.load(f)

In [139]:
# for data in result:
# #     print(type(data))
#     print(data["tokens"])

In [ ]:
# url = f"https://api.ethplorer.io/getAddressInfo/{addr}?apiKey=freekey"

# response = requests.get(url, verify=True)
# print(response.text["result"])

# https://api.etherscan.io/api?module=account&action=tokenbalance&contractaddress=0x9e1c80662d04b2c94dce0a46e5b1f5b48ce97e26&address=0x95ebe7d8089d4c8bcee0bde19c9e867cbeeefd00&tag=latest&apikey=57GBAHGQSE3CMYNZQY81V1N32YJZNCCUNZ
# url = "https://api.etherscan.io/api?module=account&action=balancemulti&address=0xddbd2b932c763ba5b1b7ae3b362eac3e8d40121a,0x63a9975ba31b0b9626b34300f7f627147df1f526,0x198ef1ec325a96cc354c7266a038be8b5c558f67&tag=latest&apikey=57GBAHGQSE3CMYNZQY81V1N32YJZNCCUNZ"